# 准备config配置文件-Mask2Former语义分割模型

同济子豪兄 2023-2-13 6-11 6-25

## 进入MMSegmentation主目录

In [1]:
import os
os.chdir('../mmsegmentation')

In [2]:
os.getcwd()

'/home/featurize/work/MMSegmentation教程20230625/mmsegmentation'

## 指定模型config配置文件

In [3]:
!rm -rf configs/mask2former/mask2former_r50_8xb2-90k_ZihaoDataset-512x1024.py # 删除原有文件

!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/watermelon/configs/mask2former_r50_8xb2-90k_ZihaoDataset-512x1024.py -P configs/mask2former

--2023-06-26 15:54:10--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/watermelon/configs/mask2former_r50_8xb2-90k_ZihaoDataset-512x1024.py
正在连接 172.16.0.13:5848... 已连接。
已发出 Proxy 请求，正在等待回应... 200 OK
长度： 6992 (6.8K) [binary/octet-stream]
正在保存至: “configs/mask2former/mask2former_r50_8xb2-90k_ZihaoDataset-512x1024.py”

mask2former_r50_8xb 100%[===================>]   6.83K  --.-KB/s    用时 0.006s  

2023-06-26 15:54:10 (1.15 MB/s) - 已保存 “configs/mask2former/mask2former_r50_8xb2-90k_ZihaoDataset-512x1024.py” [6992/6992])



## 载入config配置文件

In [4]:
from mmengine import Config
cfg = Config.fromfile('./configs/mask2former/mask2former_r50_8xb2-90k_ZihaoDataset-512x1024.py')
dataset_cfg = Config.fromfile('./configs/_base_/datasets/ZihaoDataset_pipeline.py')

## 修改config配置文件

In [5]:
# 类别个数
NUM_CLASS = 6

In [6]:
cfg.norm_cfg = dict(type='BN', requires_grad=True) # 只使用GPU时，BN取代SyncBN
cfg.crop_size = (128, 128)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg

# 模型 decode/auxiliary 输出头，指定为类别个数
cfg.model.decode_head.num_classes = NUM_CLASS

cfg.train_dataloader = dataset_cfg.train_dataloader
cfg.val_dataloader = dataset_cfg.val_dataloader
cfg.test_dataloader = dataset_cfg.val_dataloader

cfg.train_dataloader.batch_size = 4

# 结果保存目录
cfg.work_dir = './work_dirs/ZihaoDataset-Mask2Former'

# 训练迭代次数
cfg.train_cfg.max_iters = 30000
# 评估模型间隔
cfg.train_cfg.val_interval = 500
# 日志记录间隔
cfg.default_hooks.logger.interval = 100
# 模型权重保存间隔
cfg.default_hooks.checkpoint.interval = 2500

# 随机数种子
cfg['randomness'] = dict(seed=0)

## 查看完整config配置文件

In [7]:
# print(cfg.pretty_text)

## 保存最终的config配置文件

In [8]:
cfg.dump('ZihaoDataset_Mask2Former_20230626.py')